# AI Text Detection Model Evaluation

This notebook evaluates multiple models on a test dataset and compares their performance.

In [ ]:
import numpy as np
import pandas as pd
import pickle
import os
import sys
import time

sys.path.append('../')

from models.logistic_regression import LogisticRegression
from models.dnn.neuralnet import NeuralNetwork
from models.rnn.rnn import RNN
from models.rnn.optimizers import AdamOptimizer

os.makedirs('results', exist_ok=True)

## Load Dataset

In [ ]:
inputs_path = "../datasets/dataset1_inputs.csv"
outputs_path = "../datasets/dataset1_outputs.csv"

def load_csv(file_path):
    try:
        return pd.read_csv(file_path)
    except Exception as e:
        print(f"Error parsing as comma-separated: {str(e)}")
        try:
            return pd.read_csv(file_path, sep='\t')
        except Exception as e2:
            print(f"Error parsing as tab-separated: {str(e2)}")
            try:
                return pd.read_csv(file_path, encoding='latin1')
            except Exception as e3:
                print(f"Error with latin1 encoding: {str(e3)}")
                raise Exception(f"Failed to load CSV file: {file_path}")

print("Loading input dataset...")
inputs_df = load_csv(inputs_path)

print("Loading output dataset...")
outputs_df = load_csv(outputs_path)

print("Input Dataset:")
display(inputs_df.head())
print(f"Input dataset columns: {inputs_df.columns.tolist()}")

print("\nOutput Dataset:")
display(outputs_df.head())
print(f"Output dataset columns: {outputs_df.columns.tolist()}")

def identify_columns(df):
    columns = df.columns.tolist()
    
    id_col = None
    for col in columns:
        if col.lower() == 'id' or 'id' in col.lower():
            id_col = col
            break
    
    if id_col is None and len(columns) > 0:
        id_col = columns[0]
        print(f"No ID column found, using first column: {id_col}")
    
    content_col = None
    for col in columns:
        if col.lower() in ['text', 'content', 'label', 'class']:
            content_col = col
            break
    
    if content_col is None and len(columns) > 1:
        content_col = columns[1]
        print(f"No content column found, using second column: {content_col}")
    
    return id_col, content_col

input_id_col, input_text_col = identify_columns(inputs_df)
output_id_col, output_label_col = identify_columns(outputs_df)

print(f"\nUsing columns - Input: ID={input_id_col}, Text={input_text_col}")
print(f"Using columns - Output: ID={output_id_col}, Label={output_label_col}")

inputs_df = inputs_df.rename(columns={input_id_col: 'ID', input_text_col: 'Text'})
outputs_df = outputs_df.rename(columns={output_id_col: 'ID', output_label_col: 'Label'})

print("\nAfter renaming:")
print(f"Input dataset columns: {inputs_df.columns.tolist()}")
print(f"Output dataset columns: {outputs_df.columns.tolist()}")

Loading input dataset...
Error parsing as comma-separated: Error tokenizing data. C error: Expected 11 fields in line 3, saw 15

Loading output dataset...
Input Dataset:


,ID,Text
0,D1-1,"The cell cycle, or cell-division cycle, is the..."
1,D1-2,The cell cycle is the process by which a cell ...
2,D1-3,"Photons, in many atomic models in physics, are..."
3,D1-4,A photon is a fundamental particle of light an...
4,D1-5,"According to the theory of plate tectonics, Ea..."


Input dataset columns: ['ID', 'Text']

Output Dataset:


,ID\tLabel
0,D1-1\tHuman
1,D1-2\tAI
2,D1-3\tHuman
3,D1-4\tAI
4,D1-5\tHuman


Output dataset columns: ['ID\tLabel']

Using columns - Input: ID=ID, Text=Text
Using columns - Output: ID=ID	Label, Label=None

After renaming:
Input dataset columns: ['ID', 'Text']
Output dataset columns: ['ID']


## Load Models

In [ ]:
class DatasetWrapper:
    def __init__(self, X, y=None):
        self.X = X
        self.y = y

def load_model(model_type):
    model_path = f"../trained_models/{model_type}_weights.npz"
    vectorizer_path = "../preprocessed/vectorizer.pkl"
    
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file not found: {model_path}")
    if not os.path.exists(vectorizer_path):
        raise FileNotFoundError(f"Vectorizer file not found: {vectorizer_path}")
    
    if model_type == "dnn":
        model = NeuralNetwork()
        model.load(model_path)
    elif model_type == "rnn":
        model = RNN(n_units=32, input_shape=(1, 1000, 1))
        model.initialize(AdamOptimizer())
        model.load(model_path)
    else:
        model = LogisticRegression()
        model.load(model_path)

    with open(vectorizer_path, "rb") as f:
        vectorizer = pickle.load(f)
    
    return model, vectorizer

models = {}
vectorizers = {}

for model_type in ["logistic", "dnn", "rnn"]:
    try:
        model, vectorizer = load_model(model_type)
        models[model_type] = model
        vectorizers[model_type] = vectorizer
        print(f"Successfully loaded {model_type} model")
    except Exception as e:
        print(f"Error loading {model_type} model: {str(e)}")

Successfully loaded logistic model
Successfully loaded dnn model
Error loading rnn model: too many values to unpack (expected 2)


## Define Prediction Function

In [ ]:
def predict_text(text, model_type):
    if model_type not in models:
        raise ValueError(f"Model type '{model_type}' not found")
        
    model = models[model_type]
    vectorizer = vectorizers[model_type]
    
    X_new = vectorizer.transform([text]).toarray()
    
    if model_type == "dnn":
        dataset = DatasetWrapper(X_new)
        probability = model.predict(dataset)[0][0]
    elif model_type == "rnn":
        X_new_rnn = X_new.reshape((X_new.shape[0], X_new.shape[1], 1))
        predictions = model.forward_propagation(X_new_rnn, training=False)
        probability = predictions[0, -1, 0]
    else:
        probability = model.predict_proba(X_new)[0]
    
    prediction = "AI" if probability >= 0.5 else "Human"
    return prediction, probability

## Make Predictions

In [ ]:
results = {}
for model_type in models.keys():
    results[model_type] = pd.DataFrame(columns=['ID', 'Label'])

total_rows = len(inputs_df)
print(f"Making predictions on {total_rows} samples...")

start_time = time.time()
update_interval = max(1, total_rows // 20)

for index, row in inputs_df.iterrows():
    if index % update_interval == 0 or index == total_rows - 1:
        elapsed = time.time() - start_time
        progress = (index + 1) / total_rows * 100
        print(f"Progress: {progress:.1f}% ({index+1}/{total_rows}) - Elapsed: {elapsed:.1f}s")
    
    id_val = row['ID']
    text = row['Text']
    
    for model_type in models.keys():
        try:
            prediction, _ = predict_text(text, model_type)
            new_row = pd.DataFrame({'ID': [id_val], 'Label': [prediction]})
            results[model_type] = pd.concat([results[model_type], new_row], ignore_index=True)
        except Exception as e:
            print(f"Error with {model_type} model on text {id_val}: {str(e)}")
            new_row = pd.DataFrame({'ID': [id_val], 'Label': ['Error']})
            results[model_type] = pd.concat([results[model_type], new_row], ignore_index=True)

print(f"Predictions completed in {time.time() - start_time:.1f} seconds")

Making predictions on 30 samples...
Progress: 3.3% (1/30) - Elapsed: 0.0s
Progress: 6.7% (2/30) - Elapsed: 0.0s
Progress: 10.0% (3/30) - Elapsed: 0.0s
Progress: 13.3% (4/30) - Elapsed: 0.0s
Progress: 16.7% (5/30) - Elapsed: 0.0s
Progress: 20.0% (6/30) - Elapsed: 0.0s
Progress: 23.3% (7/30) - Elapsed: 0.0s
Progress: 26.7% (8/30) - Elapsed: 0.0s
Progress: 30.0% (9/30) - Elapsed: 0.0s
Progress: 33.3% (10/30) - Elapsed: 0.0s
Progress: 36.7% (11/30) - Elapsed: 0.0s
Progress: 40.0% (12/30) - Elapsed: 0.0s
Progress: 43.3% (13/30) - Elapsed: 0.0s
Progress: 46.7% (14/30) - Elapsed: 0.0s
Progress: 50.0% (15/30) - Elapsed: 0.0s
Progress: 53.3% (16/30) - Elapsed: 0.0s
Progress: 56.7% (17/30) - Elapsed: 0.0s
Progress: 60.0% (18/30) - Elapsed: 0.0s
Progress: 63.3% (19/30) - Elapsed: 0.0s
Progress: 66.7% (20/30) - Elapsed: 0.0s
Progress: 70.0% (21/30) - Elapsed: 0.0s
Progress: 73.3% (22/30) - Elapsed: 0.0s
Progress: 76.7% (23/30) - Elapsed: 0.0s
Progress: 80.0% (24/30) - Elapsed: 0.0s
Progress: 83.3%

## Save Results to CSV

In [ ]:
for model_type, df in results.items():
    output_path = f"results/{model_type}_predictions.csv"
    df.to_csv(output_path, sep='\t', index=False)
    print(f"Saved predictions for {model_type} model to {output_path}")

Saved predictions for logistic model to results/logistic_predictions.csv
Saved predictions for dnn model to results/dnn_predictions.csv


## Evaluate Model Performance

In [ ]:
outputs_df[['ID', 'Label']] = outputs_df['ID'].str.split('\t', expand=True)

print("\nAfter splitting 'ID' column:")
print(f"Output dataset columns: {outputs_df.columns.tolist()}")

def calculate_accuracy(predictions_df, ground_truth_df):
    try:
        for df, name in [(predictions_df, 'predictions'), (ground_truth_df, 'ground truth')]:
            if 'ID' not in df.columns:
                raise KeyError(f"'ID' column not found in {name} dataframe")
            if 'Label' not in df.columns:
                raise KeyError(f"'Label' column not found in {name} dataframe")
        
        merged_df = predictions_df.merge(ground_truth_df, on='ID', suffixes=('_pred', '_true'))
        
        correct = (merged_df['Label_pred'] == merged_df['Label_true']).sum()
        total = len(merged_df)
        accuracy = correct / total if total > 0 else 0
        
        return accuracy, correct, total
    except Exception as e:
        print(f"Error calculating accuracy: {str(e)}")
        print(f"Predictions columns: {predictions_df.columns.tolist()}")
        print(f"Ground truth columns: {ground_truth_df.columns.tolist()}")
        return 0, 0, 0

print("\nModel Performance:")
print("-" * 50)
for model_type, df in results.items():
    accuracy, correct, total = calculate_accuracy(df, outputs_df)
    print(f"{model_type.upper()} Model:")
    print(f"  Accuracy: {accuracy:.4f} ({correct}/{total})")
    print(f"  Percentage: {accuracy * 100:.2f}%")
    print("-" * 50)


After splitting 'ID' column:
Output dataset columns: ['ID', 'Label']

Model Performance:
--------------------------------------------------
LOGISTIC Model:
  Accuracy: 0.5000 (15/30)
  Percentage: 50.00%
--------------------------------------------------
DNN Model:
  Accuracy: 0.5667 (17/30)
  Percentage: 56.67%
--------------------------------------------------


## Detailed Analysis

In [ ]:
if len(results) > 1:
    try:
        combined_df = outputs_df[['ID', 'Label']].rename(columns={'Label': 'True_Label'})
        
        for model_type, df in results.items():
            combined_df = combined_df.merge(df, on='ID')
            combined_df = combined_df.rename(columns={'Label': f'{model_type}_pred'})
        
        pred_columns = [col for col in combined_df.columns if col.endswith('_pred')]
        combined_df['all_agree'] = combined_df[pred_columns].apply(lambda row: len(set(row)) == 1, axis=1)
        combined_df['all_correct'] = combined_df.apply(
            lambda row: row['all_agree'] and row[pred_columns[0]] == row['True_Label'], 
            axis=1
        )
        
        total = len(combined_df)
        agree_count = combined_df['all_agree'].sum()
        all_correct_count = combined_df['all_correct'].sum()
        
        print("\nModel Agreement Analysis:")
        print(f"Total samples: {total}")
        print(f"All models agree: {agree_count} ({agree_count/total*100:.2f}%)")
        print(f"All models correct: {all_correct_count} ({all_correct_count/total*100:.2f}%)")
        
        disagreement_df = combined_df[~combined_df['all_agree']]
        print(f"\nSamples where models disagree: {len(disagreement_df)}")
        display(disagreement_df.head(10))
    except Exception as e:
        print(f"Error in model agreement analysis: {str(e)}")


Model Agreement Analysis:
Total samples: 30
All models agree: 26 (86.67%)
All models correct: 14 (46.67%)

Samples where models disagree: 4


,ID,True_Label,logistic_pred,dnn_pred,all_agree,all_correct
0,D1-1,Human,AI,Human,False,False
1,D1-2,AI,AI,Human,False,False
3,D1-4,AI,Human,AI,False,False
9,D1-10,AI,Human,AI,False,False
